# SQL-Driven Reddit Data Analysis

## Introduction

**In this project, we leverage the power of SQL for Exploratory Data Analysis (EDA) using pandasql, allowing us to query and analyze Reddit data directly. Instead of relying solely on Python libraries like Pandas, we use SQL to filter, aggregate, and rank posts and authors — transforming raw data into meaningful insights.**


**This dataset contains a couple of fields with the information based on Reddit post submission**

## Data Loading and Module Setup

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)import seaborn as sns #statist graph package
import pandasql as ps

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataisbeautiful/r_dataisbeautiful_posts.csv


In [2]:
df = pd.read_csv("/kaggle/input/dataisbeautiful/r_dataisbeautiful_posts.csv")

<ipython-input-2-d1333f55545a>:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/dataisbeautiful/r_dataisbeautiful_posts.csv")


In [3]:
q = """SELECT COUNT(*) total_rows FROM df"""

print(ps.sqldf(q, locals()))

q = """SELECT * 
FROM df
LIMIT 5;
"""
print(ps.sqldf(q, locals()))

   total_rows
0      190853
       id                                              title  score  \
0  ll1p9h  Wordcloud of trending video titles on YouTube ...      1   
1  ll1o4h  Wordcloud of trending videos on YouTube in the...      1   
2  ll15gx  Immunization in India. Source: https://niti.go...      1   
3  ll0iup  How to quickly estimate the impact of players ...      1   
4  ll0g9a  How to quickly estimate the impact of players ...      1   

                 author author_flair_text        removed_by  \
0             OmarZiada             OC: 1              None   
1             OmarZiada             OC: 1         moderator   
2  Professional_Napper_              None         moderator   
3              Viziball              None  automod_filtered   
4              Viziball              None         moderator   

   total_awards_received awarders  created_utc  \
0                    0.0       []   1613473961   
1                    0.0       []   1613473829   
2               

## Post Analysis

### Most Discussed Posts

In [4]:
q = """ SELECT title, num_comments, author, full_link FROM df ORDER BY 2 DESC LIMIT 5;
"""
print(ps.sqldf(q, locals()))

                                               title  num_comments  \
0                                           data_irl         18801   
1  The best (&amp; worst) countries for raising a...         12508   
2            Police killing rates in G7 members [OC]         10425   
3  [oc] How taboo and popular are sexual fetishes...         10396   
4  Worst Episode Ever? The Most Commonly Rated Sh...          9217   

           author                                          full_link  
0      mungoflago  https://www.reddit.com/r/dataisbeautiful/comme...  
1  asherfergusson  https://www.reddit.com/r/dataisbeautiful/comme...  
2            Udzu  https://www.reddit.com/r/dataisbeautiful/comme...  
3       AellaGirl  https://www.reddit.com/r/dataisbeautiful/comme...  
4      BoMcCready  https://www.reddit.com/r/dataisbeautiful/comme...  


### The Most Upvoted Posts

In [5]:
q = """ SELECT title, score, author,full_link FROM df ORDER BY 2 DESC LIMIT 5;
"""
print(ps.sqldf(q, locals()))

                                               title   score           author  \
0  Imagine having to pay money to share your data...  116226   MOSTEPICPLAYER   
1  Let's hear it for the lurkers! The vast majori...  105255        TrueBirch   
2  Cause of Death - Reality vs. Google vs. Media ...  101451       aaronpenne   
3  Gaze and foot placement when walking over roug...   99611  sandusky_hohoho   
4                  Light Speed – fast, but slow [OC]   92686         physicsJ   

                                           full_link  
0  https://www.reddit.com/r/dataisbeautiful/comme...  
1  https://www.reddit.com/r/dataisbeautiful/comme...  
2  https://www.reddit.com/r/dataisbeautiful/comme...  
3  https://www.reddit.com/r/dataisbeautiful/comme...  
4  https://www.reddit.com/r/dataisbeautiful/comme...  


### by Reddit Rank 

The Reddit rank formula is calculated using the following formula:

$$
\text{Reddit Rank} = \frac{\text{score}}{\max(\text{score})}  + \frac{\text{comments}}{\max(\text{comments})}
$$


In [6]:
q = """
SELECT title, author, full_link,
       ((1.0 * score / ((SELECT MAX(score) FROM df))) + 
        (1.0 * num_comments / ((SELECT MAX(num_comments) FROM df)))) AS RedditRank
FROM df
ORDER BY RedditRank DESC
LIMIT 5;
"""

print(ps.sqldf(q, locals()))

                                               title          author  \
0                                           data_irl      mungoflago   
1  Let's hear it for the lurkers! The vast majori...       TrueBirch   
2  Imagine having to pay money to share your data...  MOSTEPICPLAYER   
3  Area of land burnt in Australia and area of sm...       neilrkaye   
4  Cause of Death - Reality vs. Google vs. Media ...      aaronpenne   

                                           full_link  RedditRank  
0  https://www.reddit.com/r/dataisbeautiful/comme...    1.445442  
1  https://www.reddit.com/r/dataisbeautiful/comme...    1.176549  
2  https://www.reddit.com/r/dataisbeautiful/comme...    1.055901  
3  https://www.reddit.com/r/dataisbeautiful/comme...    1.041973  
4  https://www.reddit.com/r/dataisbeautiful/comme...    1.027922  


### Counts Split by Content Maturity

In [7]:
q = """
SELECT 'Not Over 18' AS category, COUNT(*) AS count
FROM df
WHERE over_18 = 0

UNION

SELECT 'Over 18' AS category, COUNT(*) AS count
FROM df
WHERE over_18 = 1;
"""

print(ps.sqldf(q, locals()))


      category   count
0  Not Over 18  189785
1      Over 18    1068


### Post Trends Over Time 

In [8]:
q = """
SELECT 
    STRFTIME('%Y', DATETIME(created_utc, 'unixepoch')) AS year, COUNT(id) [num of posts]
FROM df
GROUP BY year
"""

print(ps.sqldf(q, locals()))

   year  num of posts
0  2012          1965
1  2013          6198
2  2014         16857
3  2015         12169
4  2016         28342
5  2017         26615
6  2018         24068
7  2019         28163
8  2020         43391
9  2021          3085


**Note: Data is based on collected Reddit posts, not a full archive of all posts per year.**

## Author Analysis

### Most Problematic Authors by Deleted Posts

In [9]:
q = """SELECT removed_by, count(distinct id) num_removed FROM df 
WHERE removed_by is not null 
GROUP BY removed_by 
ORDER BY 2 desc """

print(ps.sqldf(q, locals()))

         removed_by  num_removed
0         moderator        14789
1           deleted         2948
2  automod_filtered         1553
3            reddit         1453
4            author            1


In [10]:
q = """SELECT author, count(id) num_removed FROM df 
WHERE removed_by = 'moderator' 
GROUP BY author 
ORDER BY 2 desc 
LIMIT 10"""
print(ps.sqldf(q, locals()))

q = """SELECT AVG(num_removed) [the mean of removed posts is] FROM (SELECT author, count(id) num_removed FROM df 
WHERE removed_by = 'moderator' 
GROUP BY author 
ORDER BY 2 desc) """
print("\n",ps.sqldf(q, locals()))

                 author  num_removed
0          hornedviper9           71
1        peter_mladenov           35
2           ad55mul1994           20
3         Ashrafkamarad           20
4           Bilal078692           18
5              RushHell           17
6        Ilovedataworld           17
7                kesu11           15
8  PreferenceCreative64           15
9            Prostowned           14

    the mean of removed posts is
0                      1.344332


### Top Authors by Post Count

In [11]:
q = """ 
SELECT author, COUNT(id) [num of posts]
FROM df
GROUP BY author
ORDER BY 2 DESC 
LIMIT 5;
"""
print(ps.sqldf(q, locals()))

q = """ 
SELECT AVG([num of posts]) [Average Posts per Author]
FROM (
    SELECT author, COUNT(id) [num of posts]
    FROM df
    GROUP BY author
) AS post_counts
"""
print('\n\n', ps.sqldf(q, locals()))

       author  num of posts
0   [deleted]         26849
1   jimrosenz          1744
2  OverflowDs           446
3  caobanlong           314
4      pdwp90           300


    Average Posts per Author
0                  2.306771


### Top Scoring Authors

In [12]:
q = """ 
SELECT author, SUM(score) [total score]
FROM df
GROUP BY author
ORDER BY [total score] DESC
LIMIT 5;
"""
print(ps.sqldf(q, locals()))

q = """ 
SELECT AVG([total score]) [Average Total Score]
FROM (
    SELECT author, SUM(score) AS [total score]
    FROM df
    GROUP BY author
) AS score_counts
"""
print('\n\n', ps.sqldf(q, locals()))

               author  total score
0           [deleted]       537231
1              chartr       502180
2           neilrkaye       461362
3           datashown       430771
4  theimpossiblesalad       356430


    Average Total Score
0            406.02896


### Top Authors by Post Quality

In [13]:
q = """
SELECT author, AVG(RedditRank) 
FROM (SELECT title, author,
       ((1.0 * score / ((SELECT MAX(score) FROM df))) + 
        (1.0 * num_comments / ((SELECT MAX(num_comments) FROM df)))) AS RedditRank FROM df ORDER BY RedditRank DESC)
GROUP BY author
ORDER BY AVG(RedditRank) DESC
LIMIT 5
"""

print(ps.sqldf(q, locals()))

                author  AVG(RedditRank)
0       MOSTEPICPLAYER         1.055901
1          dinoignacio         0.770102
2   Critical_Thinking_         0.634042
3      necromanticfitz         0.630542
4  BobNelsonsBoyfriend         0.610395


### Top Authors by Total Awards Received

In [14]:
q = """ 
SELECT author, SUM(total_awards_received) AS [Total Awards]
FROM df
GROUP BY author
ORDER BY [Total Awards] DESC
LIMIT 5;
"""
print(ps.sqldf(q, locals()))

q = """ 
SELECT AVG([Total Awards]) AS [Average Total Awards]
FROM (
    SELECT author, SUM(total_awards_received) AS [Total Awards]
    FROM df
    GROUP BY author
) AS award_counts
"""
print('\n\n', ps.sqldf(q, locals()))

              author  Total Awards
0             chartr          93.0
1  already-taken-wtf          55.0
2     getToTheChopin          54.0
3   Master4pprentice          40.0
4  whereAreTheShares          35.0


    Average Total Awards
0              0.027764
